## Load environment and check for folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import subprocess
import os
base_dir = "/content/gdrive/MyDrive/final-project-paralelizacao-tecnica-de-meios-tons"
get_ipython().run_line_magic('cd', base_dir)

# assert that we have the exercise folder
assert 'repositorio' in os.listdir(), "Exercise folder not found"

In [ ]:
import os
import subprocess
import time
from datetime import datetime
from PIL import Image

# Configuração de diretórios
base_dir = os.getcwd()
exercise_dir = os.path.join(base_dir, "repositorio")
build_dir = os.path.join(exercise_dir, "build")
input_base_dir = os.path.join(exercise_dir, "img", "ppm")
output_base_dir = os.path.join(exercise_dir, "out")
log_dir = os.path.join(exercise_dir, "logs")

# Encontrar todas as imagens ppm no diretório de entrada
test_images = []
for root, dirs, files in os.walk(input_base_dir):
    for file in files:
        if file.endswith(".ppm"):
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, input_base_dir)
            test_images.append(rel_path)

Cuda de Técnicas de de Meios-Tons

In [ ]:
import os
import subprocess
import time
from datetime import datetime
from PIL import Image
import pandas as pd

# Configuração de diretórios
base_dir = os.getcwd()
exercise_dir = os.path.join(base_dir, "repositorio")
build_dir = os.path.join(exercise_dir, "build")
input_base_dir = os.path.join(exercise_dir, "img", "ppm")
output_base_dir = os.path.join(exercise_dir, "out")
log_dir = os.path.join(exercise_dir, "logs")

# Criar diretórios necessários
os.makedirs(log_dir, exist_ok=True)
os.makedirs(output_base_dir, exist_ok=True)

# Arquivos de log
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
log_filepath = os.path.join(log_dir, f"execution_log_{timestamp}.txt")
times_filepath = os.path.join(log_dir, f"execution_times_{timestamp}.csv")

# Configurações dos executáveis
executables = [
    {
        "name": "serial_c",
        "path": os.path.join(build_dir, "fsh_serial_c"),
        "command_template": "{exec_path} -i {input} -m {method} {grayscale_flag}",
        "output_suffix": "_serial_c.ppm",
        "params": {
            "method": "floyd-steinberg"  # ajuste conforme necessário
        }
    },
    {
        "name": "openmp",
        "path": os.path.join(build_dir, "fsh_omp"),
        "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
        "output_suffix": "_openmp.ppm"
    },
    {
        "name": "cuda",
        "path": os.path.join(build_dir, "fsh_cuda"),
        "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
        "output_suffix": "_cuda.ppm"
    }
]

# Parâmetros comuns
common_params = {
    "p": 0.5,
    "stochastic": 1,
    "grayscale_flag": "-g"
}

# Função para obter dimensões da imagem
def get_image_dimensions(image_path):
    try:
        with Image.open(image_path) as img:
            return img.size  # (width, height)
    except Exception as e:
        print(f"Erro ao obter dimensões de {image_path}: {str(e)}")
        return (0, 0)

# Função para registrar logs
def log_message(message, print_to_console=True):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{timestamp}] {message}\n"
    
    with open(log_filepath, 'a') as log_file:
        log_file.write(log_entry)
    
    if print_to_console:
        print(message)

# Inicializar arquivos de log
with open(log_filepath, 'w') as log_file:
    log_file.write("Log de Execução - Comparação de Implementações\n")
    log_file.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

# DataFrame para armazenar os tempos
times_data = []

# Encontrar imagens para processar
test_images = []
for root, dirs, files in os.walk(input_base_dir):
    for file in files:
        if file.endswith(".ppm"):
            rel_path = os.path.relpath(os.path.join(root, file), input_base_dir)
            test_images.append(rel_path)

log_message(f"Total de imagens encontradas: {len(test_images)}")

# Processar imagens
for i, rel_image_path in enumerate(test_images, start=1):
    img_path = os.path.join(input_base_dir, rel_image_path)
    dimensions = get_image_dimensions(img_path)
    size_str = f"{dimensions[0]}x{dimensions[1]}"
    
    log_message(f"\nProcessando imagem {i}/{len(test_images)}: {rel_image_path} ({size_str})")
    
    for exec_config in executables:
        exec_name = exec_config["name"]
        exec_path = exec_config["path"]
        
        # Criar diretório de saída específico para cada implementação
        output_dir = os.path.join(output_base_dir, exec_name, os.path.dirname(rel_image_path))
        os.makedirs(output_dir, exist_ok=True)
        
        # Nome do arquivo de saída
        output_filename = os.path.basename(rel_image_path).replace(".ppm", exec_config["output_suffix"])
        output_path = os.path.join(output_dir, output_filename)
        
        # Preparar parâmetros
        params = {**common_params, **exec_config.get("params", {})}
        params.update({
            "exec_path": exec_path,
            "input": img_path,
            "output": output_path
        })
        
        # Construir comando
        command = exec_config["command_template"].format(**params)
        
        log_message(f"  Executando {exec_name}...")
        
        # Executar e medir tempo
        start_time = time.time()
        result = subprocess.run(
            command, shell=True, cwd=exercise_dir,
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            text=True
        )
        execution_time = time.time() - start_time
        
        # Registrar resultados
        times_data.append({
            "Imagem": rel_image_path,
            "Largura": dimensions[0],
            "Altura": dimensions[1],
            "Implementacao": exec_name,
            "Tempo(s)": execution_time,
            "Output_Path": output_path
        })
        
        # Log de execução
        if result.returncode != 0:
            log_message(f"  ERRO em {exec_name}: {result.stderr}", False)
        else:
            log_message(f"  {exec_name} concluído em {execution_time:.4f}s")

# Salvar tempos em CSV
times_df = pd.DataFrame(times_data)
times_df.to_csv(times_filepath, index=False)

# Gerar relatório comparativo
report_df = times_df.groupby(['Implementacao', 'Largura', 'Altura'])['Tempo(s)'].mean().unstack(level=0)
report_filepath = os.path.join(log_dir, f"performance_report_{timestamp}.txt")

with open(report_filepath, 'w') as report_file:
    report_file.write("=== TEMPOS DE EXECUÇÃO POR IMPLEMENTAÇÃO ===\n")
    report_file.write(report_df.to_string())
    
    # Calcular speedup em relação ao serial C
    if 'serial_c' in report_df.columns:
        speedup_df = report_df.copy()
        for col in speedup_df.columns:
            if col != 'serial_c':
                speedup_df[col] = speedup_df['serial_c'] / speedup_df[col]
        
        report_file.write("\n\n=== SPEEDUP EM RELAÇÃO AO SERIAL C ===\n")
        report_file.write(speedup_df.drop(columns=['serial_c']).to_string())

log_message(f"\nProcessamento concluído. Resultados salvos em:")
log_message(f"- Log completo: {log_filepath}")
log_message(f"- Tempos de execução: {times_filepath}")
log_message(f"- Relatório comparativo: {report_filepath}")